# HW4 - Group 10

## 1) Hashing Task

In [22]:
import pandas as pd
import numpy as np
import math
import time
import sympy

we make a list of sorted passwords which we are gonna use later for finding the number of false positives 

In [24]:
sortpass = sorted(open(r"C:\Users\39335\Desktop\HW4\passwords1.txt").read().splitlines())

In [3]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords1.txt", 'r') as file :
    #df = pd.DataFrame(file.read().split("\n"), columns = ["pass1"])

In [ ]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords2.txt", 'r') as file :
    #df["pass2"] = file.read().split("\n")

In [ ]:
#len(df["pass1"])

In [ ]:
#len(df["pass2"])

for implementing bloom filter, first we need to difine indipendent hash functions. Each hash function should get a string(passwords here) as number and return an integer. We decide to define 2 hash functions and build the bloom filter based on them. In the first hash function we get the ASCII code of password by multiplying the ASCII code of each character with 37 power the position of the character. Then to get indexes from these hash numbers we calculate the modular value over the size of bit array. We decided to choose the size of bit array 9 times bigger than number of elements in the password1 file, and in order to have uniformity in our bloom filter we pick the first prime number after 9*n which is 900000011

In [25]:
def hash_fun1 (string):
    h = [ord(i) for i in string]
    hashnumber = sum([(37**h.index(i))*i for i in h])% 900000011
    return(hashnumber)
    
    

In [26]:
hash_fun1('b*ZB/egd=!XcvV>K(qY@')

784962445

In the first hash function we get the ASCII code of each password by multiplying the ASCII code of each character with 37 power the difference of  character's position and 20. To get indexes from this number, we calculate the modular value over the size of bit array

In [27]:
def hash_fun2 (string):
    h = [ord(i) for i in string]
    hashnumber = sum([(37**(20 - h.index(i)))*i for i in h])% 900000011
    return(hashnumber)

In [28]:
hash_fun2('b*ZB/egd=!XcvV>K(qY@')

290735803

The third hash function concatenate ASCII codes of each 4 characters of the password, then calculate the sum of that value and at last return the modualar value over the size of bit array

In [29]:
def hash_fun3 (string) :
    h = [ord(i) for i in string]
    hashnumber = 0
    for i in range(0,5):
        s="".join(map(str, h[4*i : 4*i + 4]))
        try :
            hashnumber += int(s)
        except :
            pass
        hashnumber = hashnumber % 900000011
        
    return(hashnumber)    

In [30]:
hash_fun3('b*ZB/egd=!XcvV>K(qY@')

80875721

The fourth hash function concatenate ASCII codes of each 5 characters of the password, then calculate the sum of that value and at last return the modualar value over the size of bit array

In [31]:
def hash_fun4 (string) :
    h = [ord(i) for i in string]
    hashnumber = 0
    for i in range(0,4):
        s="".join(map(str, h[5*i : 5*i + 5]))
        try :
            hashnumber += int(s)
        except :
            pass
        hashnumber = hashnumber % 900000011
        
    return(hashnumber)  

In [32]:
hash_fun4('b*ZB/egd=!XcvV>K(qY@')

866225930

Now, it's time to implement our bloom filter. we take an array of size m = 400,000,000 with all elements equal to 0, and for each password in password1 file we update the array

In [33]:
n = 100000000
m = (sympy.nextprime(9*n))
#print(m)
bfilter = np.zeros(m,int)

In [34]:
# this function change 0 to 1 in our bit array in indexes of 4 hashnumbers we get for each password
def update_filter(array) :
    with open(r"C:\Users\39335\Desktop\HW4\passwords1.txt", 'r') as file :
            for line in file :
                array[hash_fun1(line)] = 1
                array[hash_fun2(line)] = 1
                array[hash_fun3(line)] = 1
                array[hash_fun4(line)] = 1
   

We create a Binary search function which we are gonna use for finding the number of False positives for the bonus part. 

In [35]:
# This function will return 1 if the value exist in the array and 0 if the value does not exist
def BinarySearch(array, val):
    first = 0
    last = len(array)-1
    while (first <= last) :
        mid = (first+last)//2
        if array[mid] == val:
            return(1)
        
        elif val < array[mid]:
            last = mid -1
            
        else:
            first = mid +1
    return (0)

Now, we want to implement a search filter, which for each password check whether all 4 hash numbers exist in the bit array and if it does it would increase the counter by 1. Also we decided to do the bonus part here, so for each password in password2 file we do binary search in the sorted list of password1 to see if that password exist or not. We used binary search because it's time efficient

In [36]:
def search_filter(array) :
    counter = 0
    Truepos = 0
    with open(r"C:\Users\39335\Desktop\HW4\passwords2.txt", 'r') as file :
        for line in file :
            if(array[hash_fun1(line)] == 1  and  array[hash_fun2(line)] == 1
              and array[hash_fun3(line)] == 1 and array[hash_fun4(line)] == 1) :
                counter += 1
                Truepos += BinarySearch(sortpass, line)
                
    return(counter,Truepos)       

For deciding about the size of our bit array, we used bloom filter calculater on this website : https://hur.st/bloomfilter/ <br/>
We have the value of k and n variables, so we try different m to see which one has acceptable false positive rate for us. at last we realised that m = 9*n gives us a good rate of false positive rate, so we pick the first next prime number after it as the size of our bloom filter



In [37]:
def bloom_filter(array):
    k = 4 #number of hash functions
    n = 100000000 #elemnts in bloom filter
    m = 900000011 #size of bit array
    p = pow(1 - math.exp(-k / (m / n)), k) #probability of false positive
    start = time.time()
    update_filter(array)
    duplicates = search_filter(array)[0]
    Truepositive = search_filter(array)[1]
    end = time.time()
    runtime = end - start
    print('Number of hash function used: ', k)
    print('Number of duplicates detected: ', duplicates)
    print('Probability of false positives: ', p)
    print('Number of True duplicates is : ', Truepositive)
    print('Number of false positives are : ' , duplicates - Truepositive )
    print('Execution time: ', runtime)
    
    

In [38]:
bloom_filter(bfilter)

Number of hash function used:  4
Number of duplicates detected:  14414455
Probability of false positives:  0.016576951622835123
Number of True duplicates is :  0
Number of false positives are :  14414455
Execution time:  6891.034741163254


In [41]:
sortpass

['!"#$@wYz8=akSeyL&H4%',
 '!"#$xCctIO)lT\'S<-V+4',
 '!"#%GORX+eK)a1Dt>AT8',
 '!"#%et>X2gI8CwM(:Ec/',
 '!"#%yA,\'gl9Q-8I;.:G+',
 '!"#&w%;o36QTD.iE4FjA',
 '!"#($rJe)dMOsg%.A,ST',
 '!"#(+*4<Suj6d/ay2)G?',
 '!"#(5evt1ipFCaMl@.D8',
 '!"#(kd)YXruWa@I.tE*-',
 '!"#(r.?%xX=4GRcuQb*W',
 '!"#(zi<q710,SmE+?6IB',
 '!"#*60l:9(RmAT$B.M)p',
 '!"#,$qpGdVMr%PTD(>6J',
 '!"#,/Wo-3:u2gpH%D.1P',
 '!"#,lcvNKeGdCDYIRp.j',
 '!"#,v8/Xs.:<Ed5+SNxO',
 '!"#-=junqOeJgtr5KRH&',
 '!"#-><*RLw4Os7oAG?+c',
 '!"#-MSg:fl;BH8NqF7w0',
 '!"#-s%y<;Qwd68ET>x3p',
 '!"#.GW=)f$Jjxidghl-u',
 '!"#.Hu(&m)2KbhLNlsDZ',
 '!"#.SKWxBQ6a8-%/CT(&',
 '!"#.nS3o@br8u(9Ld760',
 '!"#/WhT-Dfd\'qx$iBwa&',
 '!"#/xR?Xc8IEJ6N,3($t',
 '!"#0$UsSTeDdgwmQa(ft',
 '!"#0GF\'k$owf)(cYmTu3',
 '!"#0SqomXUMjs.2x6*K<',
 '!"#1:pn&tC=oj,OHR@>;',
 '!"#1<qi;Ohm)F7&3R:HN',
 '!"#1?t8E2>3m=OP<gJ.@',
 '!"#1@<l.RKIQH2/cEaNx',
 '!"#1Wu/)b8yRKjUEPgh\'',
 '!"#37d24FRiBGXyb0-c%',
 '!"#40a1?6@%BI:wLGnMW',
 '!"#4jXHP;a<w6eBMF9r/',
 '!"#4y.q/ClpIRgv9AxGn',
 '!"#5&mbSe\'VoODv,d